In [17]:
import numpy as np
from sklearn.metrics import accuracy_score, log_loss

def classification_task(df):
    """
    Calculate cross-entropy loss and accuracy for a classification task.
    
    Parameters:
    df (pandas.DataFrame): DataFrame with columns for ground truth class and predicted class.
    
    Returns:
    tuple: Cross-entropy loss, accuracy
    """
    y_true = df.iloc[:, 0].values
    y_pred = df.iloc[:, 1].values
    
    # Calculate cross-entropy loss
    ce_loss = log_loss(y_true, y_pred)
    
    # Calculate accuracy
    acc = accuracy_score(y_true, np.round(y_pred))
    
    return ce_loss, acc

In [18]:
import numpy as np
from transformers import BertTokenizer, BertModel, pipeline
from nltk.translate.bleu_score import corpus_bleu
from datasets import load_metric
from transformers import LlamaForCausalLM, LlamaTokenizer

def generation_task(df):
    """
    Calculate evaluation metrics for a text generation task.
    
    Parameters:
    df (pandas.DataFrame): DataFrame with columns for original and generated text.
    
    Returns:
    dict: Dictionary containing BERT score, BLEU, GLUE, and perplexity.
    """
    original_texts = df.iloc[:, 0].tolist()
    generated_texts = df.iloc[:, 1].tolist()
    
    # Calculate BERT score
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    bert_scorer = pipeline('text-similarity', model=model, tokenizer=tokenizer)
    bert_score = np.mean([bert_scorer(orig, gen)['similarity'] for orig, gen in zip(original_texts, generated_texts)])
    
    # Calculate BLEU
    bleu = corpus_bleu([[ref] for ref in original_texts], [hyp for hyp in generated_texts])
    
    # Calculate GLUE
    glue_metric = load_metric('glue', 'stsb')
    glue_score = glue_metric.compute(predictions=generated_texts, references=original_texts)['pearson']
    
    # Calculate perplexity using LLaMA
    llama_tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
    llama_model = LlamaForCausalLM.from_pretrained("decapoda-research/llama-7b-hf")

    def calculate_perplexity(text):
        input_ids = llama_tokenizer.encode(text, return_tensors='pt')
        with torch.no_grad():
            output = llama_model(input_ids, labels=input_ids)[0]
        return torch.exp(output).item()
    
    perplexity = np.mean([calculate_perplexity(gen) for gen in generated_texts])
    
    return {
        'bert_score': bert_score,
        'bleu': bleu,
        'glue': glue_score,
        'perplexity': perplexity
    }

In [19]:
import pandas as pd
from ibm_API import get_response

In [ ]:
generation_data_df = pd.read_json("test_gen.json", encoding="utf-8")
generation_data_df["MGT"] = "MGT"

In [25]:
generation_data_df

,input,output,MGT
0,اكتب أبياتًا بعد هذا البيت: 0 اكتب أب...,دع الأمانيّ أو رُمهنّ من ظُبةٍ فإنما هنّ من غي...,MGT
1,اكتب أبياتًا بعد هذا البيت: 0 اكتب أب...,لا بل هوَ النورُ أضحى يَـــدِبُّ فـــي الظــلم...,MGT
2,اكتب أبياتًا بعد هذا البيت: 0 اكتب أب...,لما سلكوا بالهجرِ يوماً طريقتي سـيَـنْـدم بُـع...,MGT
3,اكتب أبياتًا بعد هذا البيت: 0 اكتب أب...,ما أَنتَ بِالحَكَمِ الَّذي سُمِّيتَهُ غالَتكَ ...,MGT
4,اكتب أبياتًا بعد هذا البيت: 0 اكتب أب...,إِذا المَدحُ زانَ فَتى مَعشَرٍ فَإِنَّ يَزيدَ ...,MGT
...,...,...,...
195,اكتب أبياتًا بعد هذا البيت: 0 اكتب أب...,أَجْنَى إِلَيْهَا الرِّضَى جَنَانِي مِنَ الْمَ...,MGT
196,اكتب أبياتًا بعد هذا البيت: 0 اكتب أب...,وَلَوى بِــقَــلبــي مُــذ لَوى أَصــداغــه وَ...,MGT
197,اكتب أبياتًا بعد هذا البيت: 0 اكتب أب...,وأَرسلَتِ اللَّحظَ الضَّعيفَ مع الهَوى لِيَقوى...,MGT
198,اكتب أبياتًا بعد هذا البيت: 0 اكتب أب...,ألا فـي سـبـيـل الله فـقـد أخى تقى دعـاه إلى ا...,MGT


In [6]:
for i in range(1):
    generation_data_df.iloc[i, 2] = get_response(generation_data_df.iloc[i, 0])

RESPONSE IN API FUNCTION: 
ُ 
Reshaped RESPONSE IN API FUNCTION: 
 


In [7]:
generation_data_df.iloc[10,0]

'اكتب أبياتًا بعد هذا البيت: إِن صَوَّروكَ فَإِنَّما قَد صَوَّروا تاجَ الفَخارِ وَمَطلَعَ الأَنوارِ'

In [8]:
type(generation_data_df.iloc[i, 0])

str

In [9]:
get_response(generation_data_df.iloc[0,2])

RESPONSE IN API FUNCTION: 

Reshaped RESPONSE IN API FUNCTION: 



''

In [22]:
get_response(generation_data_df.iloc[0,0])

RESPONSE IN API FUNCTION: 
 
Reshaped RESPONSE IN API FUNCTION: 
 


' '

In [11]:
generation_data_df

,input,output,MGT
0,اكتب أبياتًا بعد هذا البيت: هي المنى كثغور الغ...,دع الأمانيّ أو رُمهنّ من ظُبةٍ فإنما هنّ من غي...,ُ
1,اكتب أبياتًا بعد هذا البيت: جـسـمٌ هـوَ المـاء...,لا بل هوَ النورُ أضحى يَـــدِبُّ فـــي الظــلم...,MGT
2,اكتب أبياتًا بعد هذا البيت: فـلو عـلم الأقوامٌ...,لما سلكوا بالهجرِ يوماً طريقتي سـيَـنْـدم بُـع...,MGT
3,اكتب أبياتًا بعد هذا البيت: حَلُمَ اِبنُ قَنبَ...,ما أَنتَ بِالحَكَمِ الَّذي سُمِّيتَهُ غالَتكَ ...,MGT
4,اكتب أبياتًا بعد هذا البيت: مَدَحتُ يَزيدَ بنَ...,إِذا المَدحُ زانَ فَتى مَعشَرٍ فَإِنَّ يَزيدَ ...,MGT
...,...,...,...
195,اكتب أبياتًا بعد هذا البيت: أَكْرِمْ بِهَا مِن...,أَجْنَى إِلَيْهَا الرِّضَى جَنَانِي مِنَ الْمَ...,MGT
196,اكتب أبياتًا بعد هذا البيت: وبـــديـــع حـــسـ...,وَلَوى بِــقَــلبــي مُــذ لَوى أَصــداغــه وَ...,MGT
197,اكتب أبياتًا بعد هذا البيت: دنَت فَرأَتني عاتِ...,وأَرسلَتِ اللَّحظَ الضَّعيفَ مع الهَوى لِيَقوى...,MGT
198,اكتب أبياتًا بعد هذا البيت: تــأوهــت مــن وجـ...,ألا فـي سـبـيـل الله فـقـد أخى تقى دعـاه إلى ا...,MGT


In [23]:
generation_data_df.iloc[0,0]

'اكتب أبياتًا بعد هذا البيت:  0         اكتب أبياتًا بعد هذا البيت:  <pandas.core.stri...\n1         اكتب أبياتًا بعد هذا البيت:  <pandas.core.stri...\n2         اكتب أبياتًا بعد هذا البيت:  <pandas.core.stri...\n3         اكتب أبياتًا بعد هذا البيت:  <pandas.core.stri...\n4         اكتب أبياتًا بعد هذا البيت:  <pandas.core.stri...\n                                ...                        \n254625    اكتب أبياتًا بعد هذا البيت:  <pandas.core.stri...\n254626    اكتب أبياتًا بعد هذا البيت:  <pandas.core.stri...\n254627    اكتب أبياتًا بعد هذا البيت:  <pandas.core.stri...\n254628    اكتب أبياتًا بعد هذا البيت:  <pandas.core.stri...\n254629    اكتب أبياتًا بعد هذا البيت:  <pandas.core.stri...\nName: input, Length: 143945, dtype: object'